## 변수 범위
*해 아래 새로운 것은 없습니다.*

유효범위는 함수입니다. 함수가 끝나면 변수도 사라져요.

In [ ]:
def f1():
    v1 = 10
f1()
v1

**지역변수 -> 전역변수** 순서로 변수를 찾아갑니다.

In [ ]:
# shadowing
a = 10
def f1():
    a = 20
    print(a)
print(a)
f1()
print(a)

In [ ]:
# 왜 에러가 나지 않을까요?
def f1(a):
    print(a)
    print(b)

In [ ]:
# 실행시점에 에러가 납니다.
f1(11)

In [ ]:
# 실행시점에 b를 찾겠네요.
b = 10
f1(11)

In [ ]:
# 이상한 문제
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

try:
    f2(10)
except Exception as e:
    print(e)

- python은 line단위로 실행하는 것으로 알고 있는데?
- **실제로는 함수의 body를 통채로 compile합니다.** compile후에 한 라인씩 수행.
- 컴파일 결과 b는 local변수로 결정되었으며 print(b) 수행 시점에 local에서 b를 찾습니다.

In [ ]:
b = 6
def f2(a):
    global b  # 이게 필요합니다.
    print(a)
    print(b)
    b = 9

(참고) 내가 원하는 context에서 local, global 변수를 찾아볼 수 있습니다.

In [ ]:
b = 6
def f2(a):
    global b
    print(a)
    print(b)
    b = 9
    print(locals())
    print(globals()['b'])
f2(1)
print(globals()['b'])

## CLOSURE

함수내 함수를 정의하는 것을 단순히 closure라고 하지는 않습니다. 그렇지만 함수내 함수를 쓰면서만 closure가 발생하는 것은 맞습니다.

**확장된 변수 범위**를 갖는 **함수**를 closure라고 합니다.
- 함수 내에서 참조할 수 있지만 함수 내 선언되어 있지는 않습니다.
- 함수몸체 밖에 있는 non-global변수를 참조할 수 있습니다.
- 이 변수들을 free-variable이라고 합니다. (*global이 아니면서 local scope에 bound되지 않는 변수*)

In [ ]:
# 아직 class는 모르실 수도 있지만;
class Averager():
    def __init__(self):
        self.series = []
    def calculate(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

avg = Averager()
avg.calculate(10)

In [ ]:
# 함수로 간단히(직관적으로) 표현하고자 할 때
def make_averager():
    series = []  # free-variable이라고 합니다.
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

avg = make_averager()
avg(10)

- encapsulated된 변수범위를 포함한 함수를 표현할 때 사용합니다.
- closure 함수는 물론 class로도 표현할 수 있어요.  private 영역을 포함한 객체인스턴스를 생성하는 것과 유사하다고 볼 수 있어요.
- 함수로 표현하는게 간단할때가 있고, class에서의 self 표기법을 탈출할 수 있습니다.

함수의 free-variable을 검사할 수 있어요.

> In computer programming, the term free variable refers to variables used in a function that are neither local variables nor parameters of that function. The term non-local variable is often a synonym in this context.

https://en.wikipedia.org/wiki/Free_variables_and_bound_variables


In [ ]:
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)
print(avg.__closure__[0].cell_contents)

In [ ]:
# inner는 closure일까요?

a = 1
def outer(b):
    c = 2
    def inner(d):
        e = 10
        return d*e
    return inner

oh = outer(10)
oh(30)

In [ ]:
print(oh.__code__.co_varnames)
print(oh.__code__.co_freevars)
print(oh.__closure__)

In [ ]:
# 나는 어느 c를 보아야만 하는가
a = 1
c = 1
def outer(b):
    c = 2
    def inner(d):
        global c  # or nonlocal c로 테스트해보세요
        c = 30
        e = c*10
        return d*e
    return inner

oh = outer(10)
oh(30)

## NONLOCAL

- nonlocal == global?
- nonlocal == free-var!
- python3에서 추가된 keyword. free-variable로 유지시켜줌.

In [ ]:
b = 6
def f2(a):
    nonlocal b
    b = a
    print(b)
f2(3)
print(b)

In [ ]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

avg = make_averager()
avg(10)

In [ ]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        global count, total
        count += 1
        total += new_value
        return total / count
    return averager

avg = make_averager()
avg(10)

In [ ]:
count = 0
total = 0
def make_averager():
    def averager(new_value):
        global count, total
        count += 1
        total += new_value
        return total / count
    return averager

avg = make_averager()
avg(10)
avg(20)
count = 10
avg(30)

In [ ]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

avg = make_averager()
avg(10)
avg(20)
count = 10
avg(30)

당장 nonlocal을 어떻게 써야 하지 걱정하지 않아도 됩니다. 다른 사람의 코드를 읽을때 당황하지 않는 것부터 시작합니다.

좀 더 이해하기는 쉬었던 global자체도 유익한 것은 않습니다. 전역변수는 오랜 프로그래밍 역사동안 권장되어 오지 않았습니다.

EXERCISE

In [ ]:
# 다음 실행결과를 예측해봅시다.
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)